In [1]:
using ForwardDiff
using QuantumOptics
using LinearAlgebra
using Markdown

In [2]:
struct System{N,S1,S2,P1,P2}
    basis1::NLevelBasis{Int}
    ψ1s::Vector{S1}
    ψ2s::Vector{S2}
    proj1s::Vector{P1}
    proj1comps::Vector{P1}
    proj2s::Matrix{P2}
    proj2sub2::Matrix{P2}
    proj2oneof::Vector{P2}
    function System{N}() where N
        basis1 = NLevelBasis(N)
        ψ1s = [nlevelstate(basis1, i) for i in 1:N]
        S1 = eltype(ψ1s)
        ψ2s = [ψ1 ⊗ ψ1 for ψ1 in ψ1s]
        S2 = eltype(ψ2s)
        proj1s = [projector(ψ1) for ψ1 in ψ1s]
        P1 = eltype(proj1s)
        proj1full = sum(proj1s)
        proj1comps = [proj1full - proj for proj in proj1s]
        proj2s = [proj1 ⊗ proj2 for proj1 in proj1s, proj2 in proj1s]
        P2 = eltype(proj2s)
        proj2sub2 = [proj2s[i, i] + proj2s[i, j] + proj2s[j, i] + proj2s[j, j]
                     for i in 1:N, j in 1:N]
        proj2oneof = [proj1s[i] ⊗ proj1comps[i] + proj1comps[i] ⊗ proj1s[i] for i in 1:N]
        return new{N,S1,S2,P1,P2}(basis1, ψ1s, ψ2s, proj1s, proj1comps,
                                  proj2s, proj2sub2, proj2oneof)
    end
end

@inline function Rele(ia, ib, i, j, θ)
    if (i == ia && j == ia) || (i == ib && j == ib)
        return complex(cos(θ / 2))
    elseif (i == ia && j == ib) || (i == ib && j == ia)
        return complex(0, -sin(θ / 2))
    elseif i == j
        return complex(one(θ))
    else
        return complex(zero(θ))
    end
end

function R(sys::System{N}, ia, ib, θ) where N
    return Operator(sys.basis1, [Rele(ia, ib, i, j, θ) for i in 1:N, j in 1:N])
end
R2(sys::System, ia, ib, θa, θb) = R(sys, ia, ib, θa) ⊗ R(sys, ia, ib, θb)

function ratio_angle(r, ε=0)
    return 2 * asin(sqrt(r)) * (1 + ε)
end
R2_ratio(sys::System, ia, ib, r, εa, εb) =
    R2(sys, ia, ib, ratio_angle(r, εa), ratio_angle(r, εb))
apply(ρ, op) = op * ρ * op'

apply (generic function with 1 method)

In [3]:
struct SysWrapper{Sys,Ta,Tb}
    sys::Sys
    εa::Ta
    εb::Tb
end
@inline SysWrapper(sys, εs) = SysWrapper(sys, εs[1], εs[2])
@inline R2_ratio(sys::SysWrapper, ia, ib, r) =
    R2_ratio(sys.sys, ia, ib, r, sys.εa, sys.εb)

R2_ratio (generic function with 2 methods)

# Calibration:
First calculate how much population error is introduced by rotation angle error when driving a $\pi$ pulse. We'll use this to convert $\pi$ pulse population error into variance of rotation angle.

In [4]:
const sys2 = System{2}()
function fid_π(εs)
    sw = SysWrapper(sys2, εs)
    ψ = R2_ratio(sw, 1, 2, 1) * sys2.ψ2s[1]
    return norm(sys2.proj2s[2, 2] * ψ)^2
end
display(fid_π([0, 0]))
m = ForwardDiff.hessian(fid_π, [0.0, 0.0])
display(m)
calib_scale = m[1, 1];

1.0

2×2 Matrix{Float64}:
 -4.9348        3.70051e-32
  3.70051e-32  -4.9348

## $\pi$ pulse calibration
The $\pi$ pulse population infidelity for the two chambers are measured to be $0.5%$ and $1%$.

In [5]:
err_a = 0.5e-2;
err_b = 3.6e-2;

In [6]:
function disp_sys(sys, cb, name)
    @assert cb(sys, [0, 0]) ≈ 1
    m = ForwardDiff.hessian((x)->cb(sys, x), [0.0, 0.0])
    err_ind = m[1, 1] * (err_a + err_b) / calib_scale * 100
    err_corr = err_ind + 2 * m[1, 2] * min(err_a, err_b) / calib_scale * 100
    display(Markdown.parse("## Error for $(name)"))
    display(Markdown.parse("**Independent error: $(round(err_ind, digits=2))%**"))
    # display(Markdown.parse("**Correlated error: $(round(err_corr, digits=2))%**"))
    display(m)
end

disp_sys (generic function with 1 method)

In [7]:
@inline init_state(sw::SysWrapper{Sys} where Sys<:System{2}) =
    R2_ratio(sw, 1, 2, 1 / 2) * sw.sys.ψ2s[1]

init_state (generic function with 1 method)

In [8]:
function fid01_12(sys::System{2}, εs)
    sw = SysWrapper(sys, εs)
    ψ = init_state(sw)
    ρ = projector(ψ)
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1))
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    ρ = apply(ρ, sys.proj2sub2[1, 2])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 2])) + tr(apply(ρ, sys.proj2s[2, 1]))) / 2
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[2, 2]))) / 2
    return P_odd + c
end

fid01_12 (generic function with 1 method)

# $d=2$

In [9]:
disp_sys(sys2, fid01_12, "01 -> 12")

## Error for 01 -> 12


**Independent error: 5.12%**


2×2 Matrix{Float64}:
 -6.1685  -2.4674
 -2.4674  -6.1685

In [10]:
@inline init_state(sw::SysWrapper{Sys} where Sys<:System{3}) =
    R2_ratio(sw, 1, 3, 1 / 2) * R2_ratio(sw, 1, 2, 1 / 3) * sw.sys.ψ2s[1]

init_state (generic function with 2 methods)

In [11]:
function fid01_12(sys::System{3}, εs)
    sw = SysWrapper(sys, εs)
    ψ = init_state(sw)
    ρ = projector(ψ)
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1))
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1))
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1))
    ρ = apply(ρ, sys.proj2sub2[1, 2])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 2])) + tr(apply(ρ, sys.proj2s[2, 1]))) / 2
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[2, 2]))) / 2
    return P_odd + c
end
function fid02_10(sys::System{3}, εs)
    sw = SysWrapper(sys, εs)
    ψ = init_state(sw)
    ρ = projector(ψ)
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1))
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1))
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    ρ = apply(ρ, sys.proj2sub2[1, 2])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 2])) + tr(apply(ρ, sys.proj2s[2, 1]))) / 2
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[2, 2]))) / 2
    return P_odd + c
end
function fid12_20(sys::System{3}, εs)
    sw = SysWrapper(sys, εs)
    ψ = init_state(sw)
    ρ = projector(ψ)
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1))
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1))
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 3])) + tr(apply(ρ, sys.proj2s[3, 1]))) / 2
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[3, 3]))) / 2
    return P_odd + c
end

fid12_20 (generic function with 1 method)

# $d=3$:

In [12]:
sys3 = System{3}()
disp_sys(sys3, fid01_12, "01 -> 12")
disp_sys(sys3, fid02_10, "02 -> 10")
disp_sys(sys3, fid12_20, "12 -> 20")

## Error for 01 -> 12


**Independent error: 5.52%**


2×2 Matrix{Float64}:
 -6.64475  -1.99115
 -1.99115  -6.64475

## Error for 02 -> 10


**Independent error: 0.51%**


2×2 Matrix{Float64}:
 -0.61685  -0.61685
 -0.61685  -0.61685

## Error for 12 -> 20


**Independent error: 7.57%**


2×2 Matrix{Float64}:
 -9.11215  -1.99115
 -1.99115  -9.11215

In [13]:
@inline init_state(sw::SysWrapper{Sys} where Sys<:System{4}) =
    R2_ratio(sw, 1, 4, 1 / 2) * R2_ratio(sw, 1, 3, 1 / 3) * R2_ratio(sw, 1, 2, 1 / 4) * sw.sys.ψ2s[1]

init_state (generic function with 3 methods)

In [14]:
function fid01_12(sys::System{4}, εs)
    sw = SysWrapper(sys, εs)
    ψ = init_state(sw)
    ρ = projector(ψ)
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1))
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1))
    ρ = apply(ρ, R2_ratio(sw, 1, 4, 1))
    #
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1))
    ρ = apply(ρ, sys.proj2sub2[1, 2])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 2])) + tr(apply(ρ, sys.proj2s[2, 1]))) / 2
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[2, 2]))) / 2
    return P_odd + c
end
function fid02_13(sys::System{4}, εs)
    sw = SysWrapper(sys, εs)
    ψ = init_state(sw)
    ρ = projector(ψ)
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1))
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1))
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    ρ = apply(ρ, R2_ratio(sw, 1, 4, 1))
    #
    ρ = apply(ρ, R2_ratio(sw, 1, 4, 1))
    ρ = apply(ρ, sys.proj2sub2[1, 2])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 2])) + tr(apply(ρ, sys.proj2s[2, 1]))) / 2
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[2, 2]))) / 2
    return P_odd + c
end
function fid03_10(sys::System{4}, εs)
    sw = SysWrapper(sys, εs)
    ψ = init_state(sw)
    ρ = projector(ψ)
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1))
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1))
    ρ = apply(ρ, R2_ratio(sw, 1, 4, 1))
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    #
    ρ = apply(ρ, sys.proj2sub2[1, 2])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 2])) + tr(apply(ρ, sys.proj2s[2, 1]))) / 2
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[2, 2]))) / 2
    return P_odd + c
end
function fid12_23(sys::System{4}, εs)
    sw = SysWrapper(sys, εs)
    ψ = init_state(sw)
    ρ = projector(ψ)
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1))
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1))
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    ρ = apply(ρ, R2_ratio(sw, 1, 4, 1))
    #
    ρ = apply(ρ, R2_ratio(sw, 1, 4, 1))
    ρ = apply(ρ, sys.proj2sub2[1, 3])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 3])) + tr(apply(ρ, sys.proj2s[3, 1]))) / 2
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[3, 3]))) / 2
    return P_odd + c
end
function fid13_20(sys::System{4}, εs)
    sw = SysWrapper(sys, εs)
    ψ = init_state(sw)
    ρ = projector(ψ)
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1))
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1))
    ρ = apply(ρ, R2_ratio(sw, 1, 4, 1))
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    #
    ρ = apply(ρ, sys.proj2sub2[1, 3])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 3])) + tr(apply(ρ, sys.proj2s[3, 1]))) / 2
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[3, 3]))) / 2
    return P_odd + c
end
function fid23_30(sys::System{4}, εs)
    sw = SysWrapper(sys, εs)
    ψ = init_state(sw)
    ρ = projector(ψ)
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1))
    ρ = apply(ρ, R2_ratio(sw, 1, 3, 1))
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    ρ = apply(ρ, R2_ratio(sw, 1, 4, 1))
    ρ = 2 * apply(ρ, sys.proj2s[1, 1]) + apply(ρ, sys.proj2oneof[1])
    #
    ρ = apply(ρ, sys.proj2sub2[1, 4])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 4])) + tr(apply(ρ, sys.proj2s[4, 1]))) / 2
    ρ = apply(ρ, R2_ratio(sw, 1, 4, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[4, 4]))) / 2
    return P_odd + c
end

fid23_30 (generic function with 1 method)

# $d=4$:

In [15]:
sys4 = System{4}()
disp_sys(sys4, fid01_12, "01 -> 12")
disp_sys(sys4, fid02_13, "02 -> 13")
disp_sys(sys4, fid03_10, "03 -> 10")
disp_sys(sys4, fid12_23, "12 -> 23")
disp_sys(sys4, fid13_20, "13 -> 20")
disp_sys(sys4, fid23_30, "23 -> 30")

## Error for 01 -> 12


**Independent error: 5.84%**


2×2 Matrix{Float64}:
 -7.02764  -1.60826
 -1.60826  -7.02764

## Error for 02 -> 13


**Independent error: 0.57%**


...

**Independent error: 3.84%**


2×2 Matrix{Float64}:
 -4.61806   3.38436
  3.38436  -4.61806

## Error for 23 -> 30


**Independent error: 4.84%**


2×2 Matrix{Float64}:
 -5.82757  -2.80833
 -2.80833  -5.82757